<a href="https://colab.research.google.com/github/Pasq98/Tesi/blob/main/PrimoEsperimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Libreria Shallow learning
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import cross_val_score,cross_val_predict
from nltk.stem import PorterStemmer
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
# Import modules for evaluation purposes
# Import libraries for predcton
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc,f1_score
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, sys, getopt, pickle, csv, sklearn, re
from sklearn.model_selection import cross_validate





In [2]:
#@title Evaluate
#Funzione per valutare le prestazione di un classificatore per ogni classe
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (classification_report,
                             confusion_matrix,
                             roc_auc_score)

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
  

  
def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions)
    plt.figure(figsize=(5, 5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title("Confusion matrix (non-normalized))")
    plt.ylabel("Actual label")
    plt.xlabel("Predicted label")

In [3]:
#@title Load Dataset
def load_Cybertroll(): #Carico il dataset e ritorno il testo e i label
  CBT = pd.read_json('/content/Dataset for Detection of Cyber-Trolls.json', lines= True)
  CBT["label"] = CBT.annotation.apply(lambda x: x.get('label'))
  CBT["label"] = CBT.label.apply(lambda x: x[0])

  x = CBT['content']
  label = CBT['label']

  return x, label

def load_Formsprings():
    FS = pd.read_csv('/content/formspringPicke.csv')
    FS = FS[FS['text'].notnull()] #delete NaN
    #FS['text']=FS['text'].apply(str)
    x = FS['text']
    label = FS['label']

    return x,label

def load_Tweets(): 
  tweets=pd.read_csv('/content/Tweets.csv')
  tweets_df=tweets.drop(tweets[tweets['airline_sentiment_confidence']<0.5].index,axis=0)

  tweets_df['text'] = tweets_df['text'].str.replace('@AmericanAir', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@USAirways', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@SouthwestAir', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@JetBlue', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@VirginAmerica', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@united', '')#Rimuovo hastag iniziali

  #Rimuovo tweet con sentimento neutro
  tweets_df = tweets_df.loc[tweets_df["airline_sentiment"] != 'neutral']

  tweets_df['airline_sentiment'] = tweets_df['airline_sentiment'].str.replace('positive', '1')
  tweets_df['airline_sentiment'] = tweets_df['airline_sentiment'].str.replace('negative', '0')

  x=tweets_df['text'] #testo con caratteri speciali
  label=tweets_df['airline_sentiment']
  return x, label

def load_ACLL():
  acll=pd.read_csv('/content/ACLLTrain.csv')
  x = acll['Review']
  label = acll['Rating']
  return x, label


In [4]:
#@title Preprocessing Standard
def standard_preprocessing(text):
  clean_text = text.str.lower()
  # Remove Numbers
  clean_text = clean_text.str.replace('\d+', '')
  # Remove trailing spaces
  clean_text = clean_text.str.strip()
  # Remove Punctuations
  clean_text = clean_text.str.replace('[^\w\s]',' ')
  # Remove <br>
  clean_text = clean_text.str.replace('br', '')
  # Remove extra space in between words
  clean_text = clean_text.str.replace(' +', ' ')
  clean_text = clean_text.str.replace('\n', ' ')# per pad sequence
  # Remove Numbers
  clean_text = clean_text.str.replace('\d+', '')
  #stop word remove 
  stop = stopwords.words('english')
  clean_text = clean_text.apply(lambda x: " ".join(x for x in x.split() if x not in stop ))
  #token
  clean_text = clean_text.apply(nltk.word_tokenize)# Per il deep provo a farlo in un altro modo
  return clean_text

In [ ]:
#@title Standard con Stemming
def standardStemming(text):
  stemmer= PorterStemmer()

  clean_text = text.str.lower()
  # Remove Numbers
  clean_text = clean_text.str.replace('\d+', '')
  # Remove trailing spaces
  clean_text = clean_text.str.strip()
  # Remove Punctuations
  clean_text = clean_text.str.replace('[^\w\s]',' ')
  # Remove <br>
  clean_text = clean_text.str.replace('br', '')
  # Remove extra space in between words
  clean_text = clean_text.str.replace(' +', ' ')
  clean_text = clean_text.str.replace('\n', ' ')# per pad sequence
  # Remove Numbers
  clean_text = clean_text.str.replace('\d+', '')
  #stop word remove 
  stop = stopwords.words('english')
  clean_text = clean_text.apply(lambda x: " ".join(x for x in x.split() if x not in stop ))
  #Stemming
  clean_text = clean_text.apply(lambda x: " ".join(stemmer.stem(x) for x in x.split() ))
  #token
  clean_text = clean_text.apply(nltk.word_tokenize)# Per il deep provo a farlo in un altro modo
  return clean_text

In [ ]:
#@title Embedding
#Load glove vector
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("/content/gensim_glove_vectors.txt", binary=False)

def glove_embedding(text):
  v = glove_model.get_vector('king')
  D = v.shape[0]
  emptycount = 0
  X = np.zeros((len(text), D))
  n = 0
  for sentence in text:  
    vecs = []
    for word in sentence:
      try:
        # throws KeyError if word not found
        vec = glove_model.get_vector(word)
        vecs.append(vec)
      except KeyError:
        pass
    if len(vecs) > 0:
      vecs = np.array(vecs)
      X[n] = vecs.mean(axis=0)
    else:
      emptycount += 1
    n += 1
  print("Numer of samples with no words found: %s / %s" % (emptycount, len(text)))
  return X


In [ ]:
#@title Test STANDARD+EMBEDDING
#TEST STANDARD PREPREPROCESSING E EMBEDDING TRADITIONAL ML
x, label = load_ACLL()
x = standard_preprocessing(x)#Preprocessing
x = glove_embedding(x) #Feature extraction
#print(x)
print(x.dtype)
print(label.dtype)
label = [int(lab) for lab in label]

randomForest = RandomForestClassifier()
SVM = svm.SVC(kernel='linear')
LR = LogisticRegression(max_iter=500, class_weight='balanced')

y_pred = cross_val_predict(LR,x,label, cv=5, n_jobs=-1)
print(classification_report(label, y_pred))

In [ ]:
#@title Test STANDARD+STEMMING+EMBEDDING
x, label = load_ACLL()
x = standardStemming(x)#Preprocessing con stemming
x = glove_embedding(x) #Feature extraction

label = [int(lab) for lab in label]

randomForest = RandomForestClassifier()
SVM = svm.SVC(kernel='linear')
LR = LogisticRegression(max_iter=500, class_weight='balanced')


#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)
#print(classification_report(y_test, y_pred))

y_pred = cross_val_predict(LR,x,label, cv=5, n_jobs=-1)
print(classification_report(label, y_pred))

In [10]:
#@title get tf-idf
def get_tf_idf(text):
  # Applying TFIDF
  # You can still get n-grams here
  lower_sentences = [" ".join(j) for j in text]
  vectorizer = TfidfVectorizer(ngram_range = (1, 2),max_features=500)# ngram_range of (1, 1) means only unigrams,(1, 2) means unigrams and bigrams,and(2, 2) means only bigrams
  tf = vectorizer.fit_transform(lower_sentences)
  return tf

In [ ]:
#@title STANDARD+TF IDF
x, label = load_Cybertroll()
x = standard_preprocessing(x)#Preprocessing con stemming
x = get_tf_idf(x)
label = [int(lab) for lab in label]

randomForest = RandomForestClassifier()
SVM = svm.SVC(kernel='linear')
LR = LogisticRegression(max_iter=500, class_weight='balanced')

y_pred = cross_val_predict(LR,x,label, cv=5, n_jobs=-1)
print(classification_report(label, y_pred))